In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_csv("Weather Data (US).csv")
df.head()

,ID,DATE,TMAX,TMIN,EVAP,PRCP,Latitude,Longitude,Elevation
0,USS0012M13S,3/26/1997,NaN,NaN,NaN,0.0,37.6600,-112.7400,2920.0
1,US1OKCV0021,4/18/2007,NaN,NaN,NaN,241.0,35.1715,-97.4262,355.1
2,USC00355174,10/7/1999,NaN,NaN,NaN,NaN,42.0078,-121.3186,1410.3
3,US1MDHW0012,10/14/2015,NaN,NaN,NaN,0.0,39.3387,-76.9468,166.1
4,US1TXHRS014,10/4/2021,NaN,NaN,NaN,0.0,32.7061,-94.1683,53.6


In [ ]:
def assign_grid(df, lon_col='Longitude', lat_col='Latitude', grid_size_deg=0.5, origin_lon=None, origin_lat=None, grid_col='grid_id', return_xy=False):
    """
    Aggiunge colonne di griglia a `df` basandosi su una griglia regolare in coordinate geografiche (gradi).
    - `grid_size_deg` : dimensione della cella di griglia in gradi (es. 0.5 = mezzo grado).
    - Se `origin_lon`/`origin_lat` non forniti, la griglia è allineata al minimo di lon/lat presenti nel DF.
    Restituisce il DataFrame con colonne aggiunte: `grid_x`, `grid_y`, `grid_id`, `grid_lon`, `grid_lat`.
    Se `return_xy=True` restituisce (df, (grid_x, grid_y)).
    """
    if lon_col not in df.columns or lat_col not in df.columns:
        raise ValueError(f"Colonne richieste non trovate: {lon_col}, {lat_col}")
    lons = df[lon_col].astype(float)
    lats = df[lat_col].astype(float)
    min_lon = origin_lon if origin_lon is not None else lons.min()
    min_lat = origin_lat if origin_lat is not None else lats.min()
    x_idx = np.floor((lons - min_lon) / grid_size_deg).astype(int)
    y_idx = np.floor((lats - min_lat) / grid_size_deg).astype(int)
    df = df.copy()
    df['grid_x'] = x_idx
    df['grid_y'] = y_idx
    df[grid_col] = x_idx.astype(str) + '_' + y_idx.astype(str)
    # centro della cella
    df['grid_lon'] = min_lon + (x_idx + 0.5) * grid_size_deg
    df['grid_lat'] = min_lat + (y_idx + 0.5) * grid_size_deg
    if return_xy:
        return df, (x_idx, y_idx)
    return df

# Esempio d'uso: assegnare griglia con lato 0.5 grado e mostrare prime righe e conteggi per cella
try:
    df_with_grid = assign_grid(df, lon_col='Longitude', lat_col='Latitude', grid_size_deg=0.5)
    display(df_with_grid.head())
    print('\nConteggio eventi per grid (prime 10):')
    print(df_with_grid['grid_id'].value_counts().head(10))
except Exception as e:
    print('Errore durante l\'assegnazione delle griglie:', e)
